# Using pyspark Data-PreProcessing


In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.AssertionError: assertion failed: Expected hostname (not IP) but got 2401:4900:1c45:d38b:dd68:ce24:38d3:a2c7
	at scala.Predef$.assert(Predef.scala:170)
	at org.apache.spark.util.Utils$.checkHost(Utils.scala:1014)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:31)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:184)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:59)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:127)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:183)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:501)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newIns

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, DateType
schema = StructType([
    StructField("properties", StructType([
        StructField("parameter", StructType([
            StructField("WD10M", MapType(StringType(),DoubleType()),True),
            StructField("WS10M", MapType(StringType(),DoubleType()),True),
            StructField("WD2M",  MapType(StringType(),DoubleType()),True),
        ]), True)
    ]), True),
    StructField("geometry", StructType([
        StructField("coordinates", ArrayType(DoubleType()),True)
    ]), True)
])


city_df = spark.read.csv('file:///home/talentum/test-jupyter/Daily/program/Indian_Cities_Database.csv', header=True)

#for city in city_df.collect():
#    print(city["City"])       
# Load the json file
data_df = spark.read.json(f'file:///home/talentum/test-jupyter/Hourly/dataset/wind/Adilabad_*.json', multiLine=True, schema=schema)

#print(data_df.printSchema())
#print(data_df.show())

data_df = data_df.select(F.col("properties.parameter.*"),F.col("geometry.coordinates"))

# create dataframe for features "PS","PSC","T2M","T2MWET","T2MDEW" 
from pyspark.sql.functions import explode
WD2M =  data_df.select('coordinates',explode(data_df.WD2M).alias("Date", "WD2M"))
WD10M = data_df.select(explode(data_df.WD10M).alias("Date", "WD10M"))
WS10M = data_df.select(explode(data_df.WS10M).alias("Date", "WS10M"))
#"WD10M",WS10M, WD2M

from pyspark.sql.functions import desc
final_df = WD2M.join(WD10M, WD2M.Date == WD10M.Date, 'inner') \
.join(WS10M , WD2M.Date == WS10M.Date, 'inner') \
.select(PS.Date, "WD2M","WD10M","WS10M")

# another approach is by renaming.
# final_df.show()

# write dataframe to file in csv format
final_df.write.csv(f'file:///home/talentum/test-jupyter/Hourly/Output/Adilabad_wind', header=True)